<a href="https://colab.research.google.com/github/henry-bokyum-kim/NNStudy/blob/bokyum/%5BRL%5D%5BPG%5D%5BREINFORCE%5D%5BBK%5Dreinforce_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python3
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
import numpy as np
import random

class PGN(nn.Module):
    def __init__(self, in_size, out_size, hidden = 256):
        super(PGN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_size, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_size)
        )
    def forward(self, x):
        return self.net(x)

EPOCH = 4000
LR=0.001
GAMMA = 0.99
BATCH_EP = 3
GAME = "Acrobot-v1"

env = gym.make(GAME)
net = PGN(env.observation_space.shape[0], env.action_space.n, 128)
optim = opt.Adam(net.parameters(), lr = LR)

In [0]:
def get_onehot(i, size):
    ret= np.zeros(size)
    ret[i]=1
    return list(ret)

In [0]:
if __name__ == "__main__":    
    batch_size = 0
    batch_obs = []
    batch_act = []
    batch_qval = []
    done_ = 0
    for epoch in range(EPOCH):
        obs = env.reset()
        if GAME == "FrozenLake-v0":
            obs = get_onehot(obs, env.observation_space.n)
        cur_rew = []
        count = 0
        while True:
            with torch.no_grad():
                out = net(torch.FloatTensor(obs))
                act = np.random.choice(env.action_space.n, p=F.softmax(out).numpy())
                
            next_obs, rew, done, _ = env.step(act)
            count+=1
            
            if GAME == "FrozenLake-v0":
                if done and rew == 0:
                    rew = -200
                elif done and rew == 1:
                    rew = 500
                else:
                    rew = -4
            
            batch_obs.append(obs)
            batch_act.append(act)
            cur_rew.append(rew)
            
            obs = next_obs
            if GAME == "FrozenLake-v0":
                obs = get_onehot(obs, env.observation_space.n)
            
            if done:
                qval = []
                r_sum = 0
                for r in reversed(cur_rew):
                    r_sum*= GAMMA
                    r_sum+= r
                    qval.append(r_sum)
                batch_qval.extend(list(reversed(qval)))
                batch_size+=1
                break
        print("epoch : %d count : %d rew : %d"%(epoch, count, rew))
        if rew > 0:
            done_ +=1
                
        if batch_size == BATCH_EP:
            obss = torch.FloatTensor(batch_obs)
            acts = torch.LongTensor(batch_act).unsqueeze(-1)
            qvals = torch.FloatTensor(batch_qval).unsqueeze(-1)
                
            optim.zero_grad()
            
            logit = net(obss)
            prob = F.log_softmax(logit).gather(1, acts)
            value = qvals * prob
            
            loss = -value.mean()
            loss.backward()
            optim.step()

            batch_size = 0
            batch_obs = []
            batch_act = []
            batch_qval = []